<a href="https://colab.research.google.com/github/Warlib1975/ArduinoHttpClient/blob/master/%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_NLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Часть 1 - предсказываем букву линейной сетью

In [0]:
import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN, Concatenate, Input, concatenate, LSTM, Embedding
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

import io
import re
#from google.colab import drive
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
drive.mount('/content/drive')

NameError: ignored

In [0]:
with io.open('(Стругацкие) Тестовая_2 вместе.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') #убираем первый невидимый символ
    text = re.sub(r'[^А-я \n]', '', text) #заменяем все символы кроме кириллицы на пустые символы

In [0]:
text

'ОТЕЛЬ У ПОГИБШЕГО АЛЬПИНИСТА\n\n\n\nГЛАВА \n\n\n\n\nЯ остановил машину вылез и снял черные очки Все было так как рассказывал Згут Отель был двухэтажный желтый с зеленым над крыльцом красовалась траурная вывеска У ПОГИБШЕГО АЛЬПИНИСТА Высокие ноздреватые сугробы по сторонам крыльца были утыканы разноцветными лыжами  я насчитал семь штук одна была с ботинком С крыши свисали мутные гофрированные сосульки толщиной в руку В крайнее правое окно первого этажа выглянуло чьето бледное лицо и тут парадная дверь отворилась и на крыльце появился лысый коренастый человек в рыжем меховом жилете поверх ослепительной лавсановой рубашки Тяжелой медлительной поступью он приблизился и остановился передо мною У него была грубая красная физиономия и шея борцатяжеловеса На меня он не смотрел Его меланхолический взгляд был устремлен кудато в сторону и исполнен печального достоинства Несомненно это был сам Алек Сневар владелец отеля долины и Бутылочного Горлышка\n\n Там  произнес он неестественно низким и гл

In [0]:
#парсим текст, как последовательность символов
num_characters = 34 #33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', char_level=True) #токенизируем на уровне символов
tokenizer.fit_on_texts(text) #формируем токены на основе частотности в нашем тексте
tokenizer.word_index

{'\n': 24,
 ' ': 1,
 'а': 4,
 'б': 22,
 'в': 10,
 'г': 20,
 'д': 14,
 'е': 3,
 'ж': 26,
 'з': 19,
 'и': 6,
 'й': 25,
 'к': 12,
 'л': 9,
 'м': 13,
 'н': 5,
 'о': 2,
 'п': 16,
 'р': 11,
 'с': 8,
 'т': 7,
 'у': 15,
 'ф': 33,
 'х': 27,
 'ц': 31,
 'ч': 23,
 'ш': 28,
 'щ': 32,
 'ъ': 34,
 'ы': 18,
 'ь': 21,
 'э': 30,
 'ю': 29,
 'я': 17}

In [0]:
tokenized_text = tokenizer.texts_to_sequences('привет') #преобразуем последовательности букв в последовательности токенов
tokenized_text

[[16], [11], [6], [10], [3], [7]]

In [0]:
data = tokenizer.texts_to_matrix('привет') #последовательности токенов в OHE
data

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [0]:
''.join([tokenizer.index_word[i] for i in tokenizer.texts_to_matrix('апчхи').argmax(axis=1)]) #проверка

'апчхи'

In [0]:
data = tokenizer.texts_to_matrix(text)
n = data.shape[0]-3 #так как мы предсказываем по трем символам - четвертый, найдем последний номер первого символа
X1 = data[:n] #первый символ
X2 = data[1:n+1] #второй символ
X3 = data[2:n+2] #третий символ
Y = data[3:]

In [0]:
X1.shape

(668450, 34)

In [0]:
X1[0], X2[0], X3[0]

(array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [0]:
(Y[0]==X3[1]).all() #четвертая буква = третья буква со сдвигом на 1 = то, что мы предсказываем после первых трех букв

True

In [0]:
X1[1], X2[1], X3[1]

(array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [0]:
#готовим линейную сеть

In [0]:
input1 = Input((num_characters,), name='input_a') #вход первого символа
hidden1 = Dense(200, activation='tanh', name='hidden_a')(input1) #преобразование в скрытое пространство

dense1 = Dense(500, activation='tanh', name='dense_a')(hidden1) #представление сети о первом символе

input2 = Input((num_characters,), name='input_b') #ввод второго символа
hidden2 = Dense(200, activation='tanh', name='hidden_b')(input2) #преобразование в скрытое пространство

dense2 = Dense(500, activation='tanh', name='dense_ab')(concatenate([dense1, hidden2])) #представление сети о двух символах

input3 = Input((num_characters,), name='input_c') #ввод третьего символа
hidden3 = Dense(200, activation='tanh', name='hidden_c')(input3) #преобразование в скрытое пространство

dense3 = Dense(500, activation='tanh', name='dense_abc')(concatenate([dense2, hidden3])) #представление сети о трех символах

output4 = Dense(num_characters, activation='softmax', name='output_d')(dense3) #на основе представления о трех символах предсказываем следующий

model = Model(inputs=[input1, input2, input3], outputs=output4) #создаем модель


In [0]:
#plot_model(model)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=1e-5))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_a (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
hidden_a (Dense)                (None, 200)          7000        input_a[0][0]                    
__________________________________________________________________________________________________
input_b (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense_a (Dense)                 (None, 500)          100500      hidden_a[0][0]                   
______________________________________________________________________________________________

In [0]:
history = model.fit([X1,X2,X3], Y, batch_size=1024, epochs=200)

Epoch 1/200
653/653 [==============================] - 3s 5ms/step - loss: 3.1094 - accuracy: 0.1763
Epoch 2/200
653/653 [==============================] - 3s 5ms/step - loss: 2.7863 - accuracy: 0.2164
Epoch 3/200
653/653 [==============================] - 3s 5ms/step - loss: 2.6615 - accuracy: 0.2384
Epoch 4/200
653/653 [==============================] - 3s 5ms/step - loss: 2.5753 - accuracy: 0.2561
Epoch 5/200
321/653 [=============>................] - ETA: 1s - loss: 2.5294 - accuracy: 0.2652

In [0]:
res_str = 'при' #три символа начальной строки

a, b, c = res_str #разбиваем изначальную строку на 3 символа
str_len = 50 #сколько символов еще создать
for i in range(str_len):
    x1 =  tokenizer.texts_to_matrix(a) # преобразуем
    x2 =  tokenizer.texts_to_matrix(b) # символы в 
    x3 =  tokenizer.texts_to_matrix(c) # One-Hot-encoding
    
    pred = model.predict([x1,x2,x3]) #предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] #получаем ответ в символьном представлении
    
    res_str+=d #дописываем строку
    a,b,c = b,c,d #используем предсказание сети, как новый третий символ. остальные смещаем на 1 (второй стал первым, третий стал вторым)
    
print(res_str)

принял себя в конечно под своим с не под своим с не п


#Часть 2 - зацикливаем линейную сеть, превращаем ее в рекуррентную

In [0]:
recurrent_model = Sequential()
recurrent_model.add(Input((3, num_characters))) #при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
recurrent_model.add(SimpleRNN(500)) #рекуррентный слой на 500 нейронов
recurrent_model.add(Dense(num_characters, activation='softmax'))

recurrent_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 500)               267500    
_________________________________________________________________
dense (Dense)                (None, 34)                17034     
Total params: 284,534
Trainable params: 284,534
Non-trainable params: 0
_________________________________________________________________


In [0]:
X_rnn = np.hstack([X1,X2,X3]).reshape([X1.shape[0],3,num_characters]) #модифицируем входные данные, чтобы корректно запустить обучение

In [0]:
X_rnn[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [0]:
recurrent_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=1e-5))
recurrent_model.fit(X_rnn, Y, epochs=200, batch_size=512)

Epoch 1/200
1/1 [==============================] - 0s 1ms/step - loss: 1.8392 - accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 829us/step - loss: 1.8304 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 804us/step - loss: 1.8217 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 1ms/step - loss: 1.8129 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 1ms/step - loss: 1.8042 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 805us/step - loss: 1.7954 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 812us/step - loss: 1.7867 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 863us/step - loss: 1.7780 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 1.7692 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 874us/step - loss: 1.7605 - accuracy: 1.0000
Epoch 11/200
1/1 

In [0]:
recurrent_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=1e-6))
recurrent_model.fit(X_rnn, Y, epochs=200, batch_size=1024)

Epoch 1/200
1/1 [==============================] - 0s 1ms/step - loss: 0.4907 - accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 927us/step - loss: 0.4902 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 954us/step - loss: 0.4897 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 993us/step - loss: 0.4892 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 0.4887 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 885us/step - loss: 0.4882 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 1ms/step - loss: 0.4877 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 785us/step - loss: 0.4872 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 0.4867 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 1ms/step - loss: 0.4862 - accuracy: 1.0000
Epoch 11/200
1/1 [=

In [0]:
res_str = 'кон' #три символа начальной строки

a, b, c = res_str #разделяем их по буквам
str_len = 50 #сколько символов еще создать
for i in range(str_len):
    x1 =  tokenizer.texts_to_matrix(a) #преобразуем
    x2 =  tokenizer.texts_to_matrix(b) #символы
    x3 =  tokenizer.texts_to_matrix(c) #в OHE
    
    x123 = np.hstack([x1,x2,x3]).reshape([1,3,num_characters]) # "склеиваем три вектора в один и решейпим"
    
    pred = recurrent_model.predict(x123) #предсказываем следующий токен в формате OHE
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] #получаем из OHE номер токена и берем соответствующую букву
    
    res_str+=d #дописываем символ к строке
    a,b,c = b,c,d #делаем сдвиг на 1
    
print(res_str)

контетеветветветветветветветветветветветветветветветв


#Часть 3 - предсказываем окраску текста

##С помощью OHE

In [0]:
from tensorflow.keras.datasets import imdb #загружаем датасет отзывов на фильмы

In [0]:
num_words = 10000 #число слов, которые войдут в словарь частотности
sent_len = 140 #максимальная длина предложения (длина развернутой RNN сети)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)
print(x_train[0])
print(len(x_train[0]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
218


In [0]:
x_train = pad_sequences(x_train, maxlen=sent_len, padding='post', truncating='post') #все предложения длиной <50 дополняем нулями в конце, >50 - обрезаем
print(x_train[0])
print(x_train[0].shape)

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256]
(140,)


In [0]:
x_train = to_categorical(x_train, num_words) #превращаем токен в OHE
print(x_train[0])
print(x_train[0].shape)

In [0]:
imdb_model = Sequential()
#imdb_model.add(Dense((sent_len, 200), input_shape = (sent_len, num_words)) #Dense является одномерным слоем, но если указать две размерности, он повторит свои веса по первой оси
imdb_model.add(LSTM(200, input_shape=(sent_len, num_words))) #продвинутый RNN слой
imdb_model.add(Dense(1, activation='sigmoid')) #0- отрицательный отзыв. 1-положительный

In [0]:
imdb_model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=1e-4))

In [0]:
imdb_model.fit(x_train, y_train, batch_size=1, epochs=50, validation_split=0.2)

Epoch 1/50
625/625 [==============================] - 12s 20ms/step - loss: 0.5553 - accuracy: 0.6911 - val_loss: 0.3935 - val_accuracy: 0.8250
Epoch 2/50
625/625 [==============================] - 12s 19ms/step - loss: 0.3066 - accuracy: 0.8781 - val_loss: 0.3313 - val_accuracy: 0.8564
Epoch 3/50
625/625 [==============================] - 12s 19ms/step - loss: 0.2188 - accuracy: 0.9210 - val_loss: 0.3584 - val_accuracy: 0.8514
Epoch 4/50
625/625 [==============================] - 12s 19ms/step - loss: 0.1542 - accuracy: 0.9478 - val_loss: 0.3367 - val_accuracy: 0.8524
Epoch 5/50
625/625 [==============================] - 12s 19ms/step - loss: 0.1156 - accuracy: 0.9653 - val_loss: 0.4064 - val_accuracy: 0.8496
Epoch 6/50
625/625 [==============================] - 12s 19ms/step - loss: 0.0853 - accuracy: 0.9772 - val_loss: 0.5312 - val_accuracy: 0.8324
Epoch 7/50
625/625 [==============================] - 12s 19ms/step - loss: 0.0662 - accuracy: 0.9820 - val_loss: 0.5612 - val_accuracy:

KeyboardInterrupt: 

##С помощью Embedding

In [0]:
from tensorflow.keras.datasets import imdb #загружаем датасет отзывов на фильмы

In [0]:
num_words = 10000 #число слов, которые войдут в словарь частотности
sent_len = 140 #максимальная длина предложения (длина развернутой RNN сети)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)
print(x_train[0])
print(len(x_train[0]))

[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
218


In [0]:
x_train = pad_sequences(x_train, maxlen=sent_len, padding='post', truncating='post') #все предложения длиной <50 дополняем нулями в конце, >50 - обрезаем
print(x_train[0])
print(x_train[0].shape)

[  1  14  22  16  43 530 973   2   2  65 458   2  66   2   4 173  36 256
   5  25 100  43 838 112  50 670   2   9  35 480 284   5 150   4 172 112
 167   2 336 385  39   4 172   2   2  17 546  38  13 447]
(50,)


Embedding = OHE+Dense, но с экономией памяти. он на вход принимает номер токена

Поэтому в OHE мы не преобразовываем

In [0]:
#x_train = to_categorical(x_train, num_words) #превращаем токен в OHE
#print(x_train[0])
#print(x_train[0].shape)

In [0]:
embedding_size = 200
imdb_model = Sequential()
imdb_model.add(Embedding(num_words, embedding_size))
imdb_model.add(LSTM(200)) #продвинутый RNN слой
imdb_model.add(Dense(1, activation='sigmoid')) #0- отрицательный отзыв. 1-положительный

In [0]:
imdb_model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=1e-4))

In [0]:
imdb_model.fit(x_train, y_train, batch_size=1, epochs=50, validation_split=0.2)

Epoch 1/50
20000/20000 [==============================] - 130s 7ms/step - loss: 0.5170 - accuracy: 0.7427 - val_loss: 0.5218 - val_accuracy: 0.7374
Epoch 2/50
 3173/20000 [===>..........................] - ETA: 1:38 - loss: 0.4975 - accuracy: 0.7668

KeyboardInterrupt: ignored

#Часть 4 - перенос обучения

In [0]:
import tensorflow_hub as hub
import tensorflow as tf

In [0]:
elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=True) #загружаем слой из Tensorflow Hub

In [0]:
embed = elmo(np.array(['Какой прекрасный суп'])) #сделаем предсказание. в TF 2.0+ можно обойтись без predict - просто передаем тензор в слой, как в функцию

In [0]:
print(embed.shape)
embed #в TF 2.0+ тензоры можно выводить без сессии

(1, 1024)


<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[-0.4492077 ,  0.10092699, -0.3665941 , ..., -0.08698825,
         0.10778356, -0.24406393]], dtype=float32)>

#Часть 5 - парсим docx. XML. ElementTree. Готовим датасет для ДЗ

In [0]:
import xml.etree.ElementTree as ET

In [0]:
color_meaning = {'00ff00':'сравнение', '00ffff':'дальность', '9900ff':'эмоции', 'ff00ff':'комфорт', 'ffff00':'скорость',
                'green':'сравнение', 'yellow':'скорость', 'cyan':'дальность', 'magenta':'комфорт'} #создаем словарь соответствий цвет-значение

In [0]:
#названия необходимых тегов
r = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}r'
rpr = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}rPr'
t = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}t'
shd = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}shd'
fill = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}fill'
highlight = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}highlight'
val = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}val'

In [0]:
#парсим tesla_2
tree = ET.parse('tesla_2.xml') #открываем xml файл
root = tree.getroot() #находим корень
body = root[0]
ps2 = list(iter(body))[:2481] #получаем список всех отзывов

In [0]:
len(ps)
colors = set()

2504

In [0]:
for i, paragraph in enumerate(ps2): #проходим по всем отзывам
    phrases_list = paragraph.findall(r) #находим все фразы (часть отзыва)
    for phrase in phrases_list: #проходим по всем фразам
        text = phrase.find(t).text #получаем текст, содержащийся во фразе
        style = phrase.find(rpr) #получаем стили фразы
        
        if style.find(shd) is not None: #если размечали через заливку
            color = style.find(shd).attrib[fill] #получаем значение цвета заливки
        elif style.find(highlight) is not None: #если размечали через хайлайт
            color = style.find(highlight).attrib[val] #получаем значение цвета хайлайта
        else:
            color = 'white' #иных вариантов выделения в word нет, значит эта фраза не выделена (белый цвет)
        color = color.lower() #переводим строковое значение цвета в нижний регистр
        meaning = color_meaning[color] if color in color_meaning else '' #если есть цвет в словаре цвет-значение, то получаем значение. в противном случае у фразы значение не было выделено
        
        #colors.add(color) добавляем цвет в словарь всех встреченных цветов, если нужно проанализировать их
        
        print(text, '[{0}]'.format(meaning))

+  []
Моментальная реакция [комфорт]
,  []
Весело управляется (юркая и быстрая) [скорость]
,  []
Быстро заряжается (принимает до 250 кВт) [комфорт]
, []
 Хорошо тормозит двигателем  [комфорт]
(тормоз не нужен),  []
Постоянно обновляется ПО через интернет, Передний багажник,  [комфорт]
Привлекает взгляды [эмоции]
-  []
Центральная консоль быстро марается, Неудобные дверные ручки, Нет Apple Car Play [комфорт]
 и Android Auto []
+- Я и сам вожу модель3, правда в Калифорнии, []
 в принципе []
 машина весьма и весьма средняя, хотя дешевизна владения подкупает [эмоции]
. []
 Много минусов, но стоимость владения конечно решает. Друг предложил купить его МоделС (он брал в свое время новую за 125К), у меня был бы лайфтайм суперчардж, но при этом я бы не получил никаких кредитов и ребейтов.  []
Вобщем купил, не жалею, но и не рад. Жопу возить машина,  []
радости не приносит [эмоции]
+ Тесла топ,а не машина []
+ Машина Тесла красотка, []
для наших дорог -самое то [комфорт]
+  []
Надо брать [эмоци

15 лямов, за что? Можно купить q7 + GL + X5 [сравнение]
 и ещё жене на шубу и ребенку на велосипед останется :) []
+ Если ты директор собственной крупной IT-компании, то кандидатуры на место в твоем гараже две - либо Тесла седан, либо кроссовер.Идеальный  []
гаджет [комфорт]
!  []
Нравится [эмоции]
. Но седан нравится все таки больше)И да, скоро их будет все больше и больше, в том числе в России. []
+  []
Интересная  [эмоции]
машина)  []
+- Кузов: SUV (Кроссовер+Джип) - это что за кузов такой SUV (Кроссовер+Джип) со спойлером сзади. []
- Для фанатиков.  []
Такой бы себе никогда не взял [эмоции]
, не зависимо от своего состояния кошелька []
+  []
Авто хороший [эмоции]
, но доступный менее чем 1му % населения (причём значительно, там в промилле уже мерить надо). []
+ Это же как надо любить электрички, чтобы купить Теслу за 200 тыс. долларов? Хотя если денег много, почему нет… []
- видок довольно жалкий []
+  []
дизайн классный [эмоции]
.особенно задние двери вверх. []
- Машинка приятная 

+-  []
Жутко неудобно и непрактично [комфорт]
... и []
 морда уродливая [эмоции]
... мне из всех тесл нраится только оригинальная первая... но []
 ждать 8 часов зарядки [комфорт]
 это перебор... придумывают новые типы батарей.. пора внедрять в массы... нужно решить 2 проблемы: полная зарядка за 10 минут... и разгон после 100 км в час... и только тогда  []
можно хоронить ДВС [сравнение]
- Этот автомобиль  []
стоит в России 14-15 000 000 рублей. [эмоции]
 ПЯТНАДЦАТЬ (15!!!) МИЛЛИОНОВ РУБЛЕЙ!!! ...за автомобиль который  []
ездит по честному на 250 километров [комфорт]
,   []
едет максимум 200 км/ч [скорость]
, []
 имеет проблемы с шумоизоляцией и качеством салона [комфорт]
. Надо быть богатым гиком электромобилей, чтоб отдать за ЭТО, ТАКИЕ деньги. []
+- Все бы ни чего! но,  []
не русифицированное меню не совсем удобно [комфорт]
!!! и как путешествовать на ней, по такой стране как Россия, генератор возить с собой??? или кошки возить чтоб с придорожных линий электро передач заряжать машину?

+- сменить на lghe2, ценник резко упадет, емкость та же, токоотдача выше))) []
+  []
Отличный электромобиль [эмоции]
! []
+- Вообще то электрическая мощность автомобиля не облагается налогом и никак не влияет на ОСАГО! Значит на этот автомобиль не будет никакого транспортного налога и нет возможности посчитать ОСАГО ))) []
+ Ех... а я уже  []
годика так 2 мечтаю о таком авто [эмоции]
 =\\ Жду когда в Россию зайдут нормально и ценники скажут хотя бы оффициальные xD Кстати вопрос такой, а если ехать скажем на дальняк и сдохнут АКБ, какой нить переносной генератор дизельный смогет зарядить? Ну я так, чисто из русского мышления, залил скажем 5 литров дизеляки. он те выдал полный заряд, а то и больше и голову не надо парить с электрозаправками, а то пока они дойдут до нас =) []
+ Надо больше электромобилей, и вообще, жидкое топливо, оставить только для военной техники. []
- далеко не всем оно надо. Я лично жду пока создадут что-то городское с пробегом 200км и конкурентноспособное по ценнику

а? []
- Господа, вот объясните мне, как спасет это количество зарядок, если на заправку РЕАЛЬНО приедет 5-6 машин и будут в режиме суперчардж заряжаться? []
- Тэсла гомно! И это -3 что было бы при-20 это эпик фейл! Всю дорогу мерз в дорогой тачке чтобы отдать 5к за эвакуатор хахаха! []
- Пока не будет принципиально другой технологии аккумулятора, это  []
всё дорогие игрушки. И не особо экологичные, т.к. батарея теслы это миллион спаянных пальчиковых аккумуляторов. [комфорт]
 ДВС удобнее как ни крути [сравнение]
- Машина 👎 , а если срочно ехать-она на заряде, и заряда не хватит []
-  Зарядка никак не может стоить 60$. Максимальная стоимость зарядки в штатах 36 центов за квтч, средняя - 31 цент за квтч. В худшем случае полная зарядка обошлась бы вам в 70*0.36=25$. Если у вас вышло 60$, то вы скорее всего оставили машину на зарядной станции уже после того, как она зарядилась. Через 5 минут после того, как зарядка окончена, если вы не сняли ее с зарядки автоматом высчитывается холостой ход

In [0]:
colors #выведем все цвета, которые мы встретили

{'00FF00',
 '00FFFF',
 '8E7CC3',
 '8e7cc3',
 '9900FF',
 '9900ff',
 'B4A7D6',
 'D9EAD3',
 'EFEFEF',
 'F3F3F3',
 'F5F5F5',
 'F7F7F7',
 'F9F9F9',
 'FBFBFB',
 'FF00FF',
 'FFFF00',
 'FFFFFF',
 'b4a7d6',
 'cyan',
 'd9ead3',
 'efefef',
 'f3f3f3',
 'f5f5f5',
 'f7f7f7',
 'f9f9f9',
 'fbfbfb',
 'ff00ff',
 'ffffff',
 'green',
 'magenta',
 'white',
 'yellow'}

In [0]:
#парсим tesla_1
tree = ET.parse('tesla_1.xml')
root = tree.getroot()
body = root[0]
ps1 = list(iter(body))[:2497]

In [0]:
for i, paragraph in enumerate(ps1):
    phrases_list = paragraph.findall(r)
    for phrase in phrases_list:
        text = phrase.find(t).text
        style = phrase.find(rpr)
        
        if style.find(shd) is not None:
            color = style.find(shd).attrib[fill]
        elif style.find(highlight) is not None:
            color = style.find(highlight).attrib[val]
        else:
            color = 'white'
        color = color.lower()
        meaning = color_meaning[color] if color in color_meaning else ''
        
        print(text, '[{0}]'.format(meaning))

+ Да, её  []
угонять страшно [комфорт]
, машина сама тебя отвезет к отделу полиции или ГИБДД! []


In [0]:
##xtrain - ytrain

##Подготовка датасета для ДЗ

In [0]:
xtrain1 = [] #словарь отзывов по первому документу
ytrain1 = [] #словарь эмоциональных окрасок отзывов

In [0]:
for i, paragraph in enumerate(ps1): #проходим по первому документу
    phrases_list = paragraph.findall(r) #находим все фразы отзыва
    text='' #заготовка под текст
    for phrase in phrases_list: #проходим по всем фразам
        text+= phrase.find(t).text #дописываем к заготовке очередную фразу
    if text == '': print(i+1) #если ничего не дописали, выведем номер отзыва
    
    if text[:2] == '+-': #если отзыв начинается с +-
        sentiment = 0 #то он нейтральный
        text = text[2:] #убираем +- с начала отзыва  -------\
    elif text[0] == '+': #если отзыв начинается с +          \
        sentiment = 1 #то он позитивный                       \
        text = text[1:] #убираем - с начала отзыва   ------------> не является обязательным. токенизатор все равно все уберет
    elif text[0] == '-': #если отзыв начинается с -           /  
        sentiment = -1 #то он негативный                     /
        text = text[1:] #убираем - с начала отзыва   -------/
    else: #если не начинается с +, - или +-
        sentiment = 'ERROR' #то отзыв размечен неверно
        
    xtrain1.append(text) #записываем полный отзыв
    ytrain1.append(sentiment) #записываем окраску

In [0]:
for i in range(len(ytrain1)): #проходим по окраскам
    if ytrain1[i]=='ERROR': #если была ошибка в разметке
        print(i+1) #выведем номер отзыва

In [0]:
xtrain2 = []
ytrain2 = []

In [0]:
for i, paragraph in enumerate(ps2):
    phrases_list = paragraph.findall(r)
    text=''
    for phrase in phrases_list:
        text+= phrase.find(t).text
    if text == '': print(i)
    
    if text[:2] == '+-':
        sentiment = 0 #neutral
        text = text[2:]
    elif text[0] == '+':
        sentiment = 1 #positive
        text = text[1:]
    elif text[0] == '-':
        sentiment = -1 #negative
        text = text[1:]
    else:
        sentiment = 'ERROR'
        
    xtrain2.append(text)
    ytrain2.append(sentiment)

In [0]:
for i in range(len(ytrain2)):
    if ytrain2[i]=='ERROR':
        print(i)

In [0]:
X_train = xtrain1 + xtrain2 #соединяем результат
Y_train = ytrain1 + ytrain2 #обработки двух документов

In [0]:
#посмотрим отзыв и его окраске по номеру
n = 35
print(X_train[n])
print(Y_train[n])

 Образно говоря - мой IPS дисплей на ноутбуке со светодиодной подсветкой и разрешением 2048*1536 точек с соотношением сторон 4:3 настолько офуительно хорош, что мне похер на чуть отходящую рамку корпуса справа сзади, я уже три года все хочу ее поправить - и не делаю этого, потому что это настолько вторично, что словами не передать…
1
